# Assignment 8
### Joseph Annand
### 8/25/2023

### The libraries you will use are already loaded for you below

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import chain

## Question 1
Read in the two Netflix CSV files from /Data/Netflix as pandas dataframes. Print the number of unique genres. This is not as simple as it sounds. You cannot simply find the length of `titles['genres'].unique()`. You must convert the output of that code to a list, iterate over that list and replace the following characters: `[]',`. Once you have them replace you can split the individual strings to list items and flatten the list. I have already imported the `chain()` function for you to flatten the list. Look up the documentation to see its usage. There are 19 unique genres, but I want you to write the code to find them. 

In [ ]:
# your code here
titles_data = pd.read_csv("Data/Netflix/titles.csv")
credits_data = pd.read_csv("Data/Netflix/credits.csv")

genres = list(titles_data['genres'].unique())

for i in range(len(genres)):
    genres[i] = genres[i].replace("[", "").replace("]", "").replace(",", "").replace("'", "")
    genres[i] = genres[i].split()

genres = list(chain.from_iterable(genres))
genres = pd.Series(genres).unique()
print(genres.size)

## Question 2
Print the release year and the imdb score of the highest average score of all movies by year. This is trickier than it sounds. To do this you will need to aggregate the means by year. If you use the simple method you will get a pandas series. The series will need to be converted to a dataframe and the index will need to be set as a column (release year). Once you have done that you can find the numerical index with the highest average imdb score. 

In [ ]:
# your code here
movies = titles_data[titles_data['type'] == "MOVIE"]

movies_year = movies.groupby(by="release_year")['imdb_score'].max()

highest_rated = pd.DataFrame(movies_year)

print(highest_rated.idxmax())

## Question 3
There were 208 actors in the movie with the most credited actors. What is the title of that movie? Nulls and NaN values do not count. 

In [ ]:
# your code here
acting_credits = pd.DataFrame(credits_data.groupby(by='id')['name'].count())
movie_id = acting_credits['name'].idxmax()
print(pd.DataFrame(titles_data.loc[titles_data['id'] == movie_id, 'title']))

## Question 4
Which movie has the highest IMDB score for the actor Robert De Niro? What year was it made? Create a kdeplot (kernel density estimation to show the distribution of his IMDB movie scores. 

In [ ]:
# your code here
de_niro_credits = pd.DataFrame(credits_data.loc[credits_data['name'] == "Robert De Niro", 'id'])

de_niro_id = list(de_niro_credits['id'])

de_niro_movies = pd.DataFrame(titles_data.loc[titles_data['id'].isin(de_niro_id), :])

print(de_niro_movies.at[de_niro_movies['imdb_score'].idxmax(), 'title'])
print(de_niro_movies.at[de_niro_movies['imdb_score'].idxmax(), 'release_year'])

sns.kdeplot(data=de_niro_movies, x="imdb_score")

## Question 5
Create two new boolean columns in the titles dataframe that are true when the description contains war or gangster. Call these columns `war_movies` and `gangster_movies`. How many movies are there in both categories? Which category has a higher average IMDB score? Show the IMDB score kernel density estimations of both categories.


In [ ]:
# your code here
titles_data['war_movies'] = np.where(titles_data['description'].str.contains("war"), True, False)
titles_data['gangster_movies'] = np.where(titles_data['description'].str.contains("gangster"), True, False)

print(len(titles_data[titles_data['war_movies'] == True]))
print(len(titles_data[titles_data['ganster_movies'] == True]))

war_movie_data = titles_data.loc[titles_data['war_movies'] == True, :]
gang_movie_data = titles_data.loc[titles_data['gangster_movies'] == True, :]

war_score = np.mean(war_movie_data['imdb_score'])
gangster_score = np.mean(gang_movie_data['imdb_score'])
if  (war_score > gangster_score):
    print(f"Average war movie IMdB score of {war_score} is higher than average gangster movie score of {gangster_score}")
else:
    print(f"Average ganster movie IMdB score of {gangster_score} is higher than average war movie score of {war_score}")

sns.kdeplot(data=war_movie_data, x='imdb_score')
sns.kdeplot(data=gangster_score, x='imdb_score')